In [10]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq
import string
from nltk.corpus import stopwords 


In [9]:


def remove_punctuation(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s
#TODO: fix this or dont use
def get_longword(s, n):
    s = re.split(' ', s)
    s =  [w for w in s if not w in stops]
    return heapq.nlargest(n, s, key=len)

def get_rarewrd(s):
    s = re.split(' ', s)
    s =  [w for w in s if not w in stops]
    freqs = [words.count(x) for x in s]
    dd= [i for i,x in enumerate(freqs) if x == min(freqs)]
    return [s[dd[0]]]

#TODO: make this work with one word/period
def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a
    
def run_through_theta(v):
    return v.dot(theta.as_matrix())

def get_winner_from_avecs(row):
	try:	
		dists = []
		for col in row['answerA':'answerD']:
		#TODO: if 'not' in words: 1-avec
			avec = get_avg_vec(col)
			#if 'not' in re.split(' ',col):
			#	avec = 1-avec
			#	print('NOT')
			#why nan errors?
			if sum(np.isfinite(avec)) < 300:
				print('avec')
				print(avec)
			if  sum(np.isfinite(row['pvec'])) < 300:
				print('pvec')
				print(row['pvec'])
			dist = cosine(avec, row['pvec'])
			dists.append(dist)
		m = np.nanmin(dists)
		best = [i for i, j in enumerate(dists) if j == m]
		if best == [0]:
			return 'A'
		if best == [1]:

			return 'B'
		if best == [2]:
			return 'C'
		if best == [3]:
			return 'D'
		else:
			print('returning C becuase no best cosine')
			print(len(best))
			print(dists)
			print(m)
			return 'C'
	except:
		print('some error, returning C')
		return 'C'
    
    
    
    
def word_clusters(row):
    if str(row.id)[-2:] == '01':
        print(row.id)
        print('\n\n\n\n')
    #print(row.correctAnswer)
    try:
        keywds = row.keyword

        word_pos = {}
        for keyw in keywds:
            ls = [i for i,x in enumerate(words) if x == keyw]
            #print(ls)
            word_pos[keyw] = ls

        spots = []
        for key1 in keywds:
            print(key1)
            matches = {}
            for z in word_pos[key1]:
                dists=[]
                for key_ in keywds:
                    d= abs(min(word_pos[key_], key=lambda x:abs(x-z))-z)
                    dists.append(d)
                #print(z)
                #print(dists)
                dists.remove(max(dists))
                matches[z] = sum(dists)

            zzz= [k for k, v in matches.items() if v ==min(matches.values())]
            spots.append(zzz[0])
        print(spots)
        answers = row['akeyword':'dkeyword']
        kkk = []
        for answer in answers:
            print(answer[0])
            waka = 0
            for x in spots:
                wd = words[x-1000:x+1000]
                ct = wd.count(answer[0])
                waka = (waka + ct)#/(words.count(answer[0])/2)    #trying to skew for 
            kkk.append(waka)
        mx = max(kkk)
        print(kkk)
        best = [i for i, j in enumerate(kkk) if j == mx]
        if best == [0]:
            return 'A'
        if best == [1]:
            return 'B'
        if best == [2]:
            return 'C'
        if best == [3]:
            return 'D'
        else:
            print('returning C becuase no max')
            return 'C'
    except:
        print('returning C becuase of BIG PROBLEM')
        return 'C'
    

In [3]:
#data  = pd.read_csv('../input/training_set.tsv', '\t')
data  = pd.read_csv('../input/validation_set.tsv', '\t')

In [20]:
stop = stopwords.words('english')

In [21]:

file = open('../input/Concepts.txt', 'r')
words = list(file.read().split())
words = [word.strip(string.punctuation) for word in words]
words = [w for w in words if not w in stop]

In [5]:
data.question = data.question.apply(remove_punctuation)
data.answerA = data.answerA.apply(remove_punctuation)
data.answerB = data.answerB.apply(remove_punctuation)
data.answerC = data.answerC.apply(remove_punctuation)
data.answerD = data.answerD.apply(remove_punctuation)

In [6]:
data['keyword'] =data['question'].apply(get_longword, args = (4,))

#I get to choose this
#data['qvec'] = data.keyword.apply(get_avg_vec) 

In [ ]:
data['akeyword'] =data['answerA'].apply(get_rarewrd)
data['bkeyword'] =data['answerB'].apply(get_rarewrd)
data['ckeyword'] =data['answerC'].apply(get_rarewrd)
data['dkeyword'] =data['answerD'].apply(get_rarewrd)


In [ ]:
data['guess'] = data.apply(word_clusters, axis = 1)

102501





developing
industry
examples
process
[706424, 260638, 706465, 706423]
during
yard
nervous
of
[0, 0, 0, 0]
returning C becuase no max
reproduction
temperature
recycling
directly
[496158, 496001, 138433, 138470]
panting
urethra
predators
water
[0, 0, 4, 24]
framework
provides
better
organs
[531570, 531592, 514206, 531597]
muscular
in
and
tissue
[0, 0, 0, 26]
hermaphroditic
growing
season
Which
[54315, 483107, 483108, 483335]
coniferous
no
there
perennial
[3, 0, 0, 0]
marsupials
following
transport
response
[76104, 689293, 689301, 689201]
heart
glucagon
hatched
to
[0, 0, 0, 0]
returning C becuase no max
deficiency
returning C becuase of BIG PROBLEM
organisms
produces
studying
ancient
[144133, 639000, 144152, 144132]
Grain
pottery
ridding
circulatory
[0, 0, 0, 0]
returning C becuase no max
describes
following
hepatitis
nitrogen
[816557, 816571, 546637, 344753]
thymineadenine
the
slime
and
[0, 0, 0, 0]
returning C becuase no max
vaccination
statement
impact
living
[501609, 48990

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')
sample['correctAnswer'] = data['guess']
sample.to_csv('../output/word_cluster_00.csv', index=False)

In [9]:
data.head()

,id,question,answerA,answerB,answerC,answerD,keyword,qvec,akeyword,bkeyword,ckeyword,dkeyword,aavec,bavec,cavec,davec
0,102501,A meter is industry by developing a process of...,cool foods during shipping,yard,nervous system,birds of prey,"[developing, industry, examples, process, heat...","[-0.0076541421935, 0.0428126163781, -0.0093698...",[shipping],[yard],[nervous],[birds],"[0.0599160343409, 0.123002231121, -0.045650314...","[0.00865275319666, 0.0433680191636, 0.04642602...","[0.0429564490914, 0.00291655748151, -0.0250321...","[0.0218830313534, 0.0545616932213, -0.10153727..."
1,102502,Which part of directly involved in recycling t...,panting,urethra,predators,water,"[reproduction, temperature, recycling, directl...","[-0.00762585951015, 0.00861827973276, -0.01647...",[panting],[urethra],[predators],[water],"[0.0639518499374, 0.0659503489733, -0.02526669...","[-0.0568876340985, 0.0671530738473, -0.1180525...","[-0.0123091973364, 0.0607602931559, -0.1267585...","[-0.0583994314075, 0.0527478717268, 0.04502407..."
2,102503,Which of the seeds grow better in the light or...,muscular system,keep one group in a light place and another in...,skates and rays,nervous tissue,"[framework, provides, better, organs, Which]","[-0.0406858625356, 0.000434694252908, 0.027284...",[muscular],[another],[skates],[nervous],"[0.0851534381509, 0.12938041985, 0.03531557321...","[0.119020909071, -0.0120366625488, 0.056220933...","[-0.0518872700632, -0.0753730908036, 0.0167950...","[0.0429564490914, 0.00291655748151, -0.0250321..."
3,102504,Which of the hermaphroditic This growing seaso...,coniferous forest,no reproductive organs,there is less water loss through small leaves,perennial,"[hermaphroditic, growing, season, Which, think]","[-0.0035101480782, 0.0597439706326, -0.0213354...",[coniferous],[reproductive],[through],[perennial],"[0.0473578386009, 0.0444385223091, 0.017191544...","[0.00988108664751, 0.0588776506484, -0.0523356...","[0.00783920008689, 0.0335110835731, -0.0832989...","[0.0817415639758, 0.203823119402, 0.0391014851..."
4,102505,When marsupials following is not a transport o...,heart,glucagon,hatched,to protect vital organs,"[marsupials, following, transport, response, f...","[0.00498434053734, 0.000841771054547, -0.01591...",[heart],[glucagon],[hatched],[protect],"[-0.0549635440111, 0.0888696238399, 0.02337735...","[-0.0631601214409, -0.0484089106321, -0.010907...","[-0.0236770678312, 0.0858293697238, -0.0198295...","[-0.0634209290147, 0.0630781129003, -0.0521080..."
